<a href="https://colab.research.google.com/github/ShalinPatel670/ShalinPatel670/blob/main/SHAISH_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Detects simple emotions from the facial expression images.

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

train_dir = r"C:\Users\25sha\Downloads\fer2013_jpg_format.zip\train"

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    class_mode="categorical",
    color_mode="grayscale",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    class_mode="categorical",
    color_mode="grayscale",
    subset="validation"
)

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(7, activation='softmax')  # 7 emotion classes
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)

model.save("emotion_model.h5")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()


In [ ]:
# Detects which of the 8 simple emotions any voice is giving.

import os
import librosa
import librosa.display
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization

ravdess_path = r"C:\Users\25sha\Downloads\archive\audio_speech_actors_01-24"

file_emotion = []
file_path = []

ravdess_directory_list = os.listdir(ravdess_path)
for dir in ravdess_directory_list:
    actor_folder = os.path.join(ravdess_path, dir)
    if os.path.isdir(actor_folder):
        for file in os.listdir(actor_folder):
            part = file.split('.')[0].split('-')
            if len(part) > 2:
                emotion_label = int(part[2])
                file_emotion.append(emotion_label)
                file_path.append(os.path.join(actor_folder, file))

emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)

emotion_map = {
    1: 'neutral', 2: 'calm', 3: 'happy', 4: 'sad',
    5: 'angry', 6: 'fear', 7: 'disgust', 8: 'surprise'
}
ravdess_df.Emotions.replace(emotion_map, inplace=True)

plt.figure(figsize=(8,5))
sns.countplot(x=ravdess_df["Emotions"])
plt.xlabel("Emotion")
plt.ylabel("Count")
plt.title("Distribution of Emotions in RAVDESS Dataset")
plt.show()

def extract_features(file_path, max_length=500):
    y, sr = librosa.load(file_path, sr=16000)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

    if mfccs.shape[1] > max_length:
        mfccs = mfccs[:, :max_length]
    else:
        pad_width = max_length - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    return mfccs

X = np.array([extract_features(f) for f in ravdess_df["Path"]])
X = X[..., np.newaxis]

unique_emotions = list(emotion_map.values())
y = np.array([unique_emotions.index(emotion) for emotion in ravdess_df["Emotions"]])
y = to_categorical(y, num_classes=len(unique_emotions))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(64, kernel_size=3, activation="relu", input_shape=(40, 500, 1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(len(unique_emotions), activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

model.save("/mnt/data/ravdess_emotion_model.h5")

plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
# Encrypting facial expressions to avoid identification and ensure privacy.

import os
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import cv2
import numpy as np

dataset_path = "dataset"
encrypted_dataset_path = "encrypted_dataset"

for split in ["train", "test"]:
    split_path = os.path.join(encrypted_dataset_path, split)
    if not os.path.exists(split_path):
        os.makedirs(split_path)

key_path = "encryption_key.bin"

if not os.path.exists(key_path):
    encryption_key = get_random_bytes(32)
    with open(key_path, "wb") as key_file:
        key_file.write(encryption_key)
else:
    with open(key_path, "rb") as key_file:
        encryption_key = key_file.read()

def encrypt_image(image_path, key):
    image = cv2.imread(image_path)
    _, img_encoded = cv2.imencode('.png', image)
    image_bytes = img_encoded.tobytes()

    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(image_bytes)

    return cipher.nonce + ciphertext

for split in ["train", "test"]:
    for emotion_folder in os.listdir(os.path.join(dataset_path, split)):
        folder_path = os.path.join(dataset_path, split, emotion_folder)
        encrypted_folder_path = os.path.join(encrypted_dataset_path, split, emotion_folder)

        if not os.path.exists(encrypted_folder_path):
            os.makedirs(encrypted_folder_path)

        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            encrypted_data = encrypt_image(image_path, encryption_key)

            encrypted_image_path = os.path.join(encrypted_folder_path, image_file + ".enc")
            with open(encrypted_image_path, "wb") as enc_file:
                enc_file.write(encrypted_data)

print("Dataset Encryption Complete! All images are encrypted.")

In [ ]:
# Decrypting so only those with the keys can even open it-- but in any case,
# the file will just go straight for use in the model.

import os
from Crypto.Cipher import AES
import cv2
import numpy as np

encrypted_dataset_path = "encrypted_dataset"
decrypted_dataset_path = "decrypted_dataset"
key_path = "encryption_key.bin"

for split in ["train", "test"]:
    split_path = os.path.join(decrypted_dataset_path, split)
    if not os.path.exists(split_path):
        os.makedirs(split_path)

with open(key_path, "rb") as key_file:
    encryption_key = key_file.read()

def decrypt_image(encrypted_image_path, key):
    with open(encrypted_image_path, "rb") as file:
        encrypted_data = file.read()

    nonce = encrypted_data[:16]
    ciphertext = encrypted_data[16:]

    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    decrypted_data = cipher.decrypt(ciphertext)

    np_arr = np.frombuffer(decrypted_data, np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    return img

for split in ["train", "test"]:
    for emotion_folder in os.listdir(os.path.join(encrypted_dataset_path, split)):
        folder_path = os.path.join(encrypted_dataset_path, split, emotion_folder)
        decrypted_folder_path = os.path.join(decrypted_dataset_path, split, emotion_folder)

        if not os.path.exists(decrypted_folder_path):
            os.makedirs(decrypted_folder_path)

        for image_file in os.listdir(folder_path):
            encrypted_image_path = os.path.join(folder_path, image_file)
            decrypted_img = decrypt_image(encrypted_image_path, encryption_key)
            decrypted_image_path = os.path.join(decrypted_folder_path, image_file.replace(".enc", ".png"))
            cv2.imwrite(decrypted_image_path, decrypted_img)

print("Decryption Complete! Images restored.")

In [ ]:
# Circumplex flood. Every emotion is mapped using the circumplex model,
# and can be vectorized with (valence, arousal) [valence is positive/negative, arousal is low-energy/high-energy.]


import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# mapping the emotions using the circumplex model...
emotion_mapping = {
    "angry": (-0.7, 0.7),
    "disgust": (-0.7, 0.5),
    "fear": (-0.6, 0.8),
    "happy": (0.7, 0.7),
    "neutral": (0.0, 0.0),
    "sad": (-0.8, -0.6),
    "surprise": (0.6, 0.8)
}

complex_emotion_mapping = {
    "Distressed": (-0.4, 0.6),
    "Tense": (-0.3, 0.4),
    "Alert": (0.1, 0.5),
    "Excited": (0.5, 0.6),
    "Content": (0.4, 0.3),
    "Depressed": (-0.5, -0.5),
    "Bored": (-0.2, -0.4),
    "Neutral": (0.0, 0.0)
}

simple_emotions = list(emotion_mapping.keys())
complex_emotions = list(complex_emotion_mapping.keys())

# Using synthetic simple emotions, using arclength on circumplex model to derive complex emotions.
# Less arclength between simple and complex emotion = stronger weight, as the two are more intertwined.
# This simulates how simple emotions form complex ones in a natural way.

def compute_weights():
    weights = np.zeros((len(simple_emotions), len(complex_emotions)))
    for i, simple in enumerate(simple_emotions):
        for j, complex_ in enumerate(complex_emotions):
            v1, a1 = emotion_mapping[simple]
            v2, a2 = complex_emotion_mapping[complex_]
            arclength = np.arccos((v1 * v2 + a1 * a2) / (np.sqrt(v1**2 + a1**2) * np.sqrt(v2**2 + a2**2) + 1e-6))
            weights[i, j] = 1 / (arclength + 0.1)
    weights /= weights.sum(axis=1, keepdims=True)
    return weights

weights_matrix = compute_weights()

num_samples = 200
X_synthetic = np.random.dirichlet(np.ones(len(simple_emotions)) * 3, size=num_samples)
y_synthetic_probs = np.dot(X_synthetic, weights_matrix)
y_synthetic_labels = np.argmax(y_synthetic_probs, axis=1)
y_synthetic_emotions = [complex_emotions[label] for label in y_synthetic_labels]

df_synthetic = pd.DataFrame(X_synthetic, columns=simple_emotions)
df_synthetic["Complex Emotion"] = y_synthetic_emotions
y_synthetic_encoded = np.array([complex_emotions.index(emotion) for emotion in y_synthetic_emotions])

# deep learning uses the synthetic dataset involving the simple and derived complex emotions.

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(len(simple_emotions),)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(complex_emotions), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_synthetic, y_synthetic_encoded, epochs=50, batch_size=16, verbose=1)

test_samples = 50
X_test = np.random.dirichlet(np.ones(len(simple_emotions)) * 3, size=test_samples)
y_pred_probs = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_probs, axis=1)
predicted_complex_emotions = [complex_emotions[label] for label in y_pred_labels]

# plotting data.

df_predictions = pd.DataFrame(X_test, columns=simple_emotions)
df_predictions["Predicted Complex Emotion"] = predicted_complex_emotions

counts = pd.Series(predicted_complex_emotions).value_counts().reindex(complex_emotions, fill_value=0)

plt.figure(figsize=(10, 6))
counts.plot(kind="bar", color="purple", edgecolor="black")
plt.xlabel("Predicted Complex Emotion")
plt.ylabel("Frequency")
plt.title("Distribution of Predicted Complex Emotions")
plt.xticks(rotation=45)
plt.show()



In [ ]:
# Used botpress to make an LLM trained on research and other info on crisis situations and therapy. It

<script src="https://cdn.botpress.cloud/webchat/v1/inject.js"></script>
<script>
  window.botpressWebChat.init({
      "composerPlaceholder": "Chat with your bot",
      "botConversationDescription": "I'm here to assist you!",
      "botId": "7416bbf4-4223-4490-9169-01c5a3f2ae7a",
      "hostUrl": "https://cdn.botpress.cloud/webchat/v1",
      "messagingUrl": "https://messaging.botpress.cloud/",
      "clientId": "7416bbf4-4223-4490-9169-01c5a3f2ae7a",
      "webhookId": "8baf76d9-c0e8-4718-8005-b23c093b8543",
      "lazySocket": true,
      "themeName": "prism",
      "botName": "YourBot",
      "frontendVersion": "v1",
      "useSessionStorage": true,
      "theme": "prism",
      "themeColor": "#2563eb",
      "allowedOrigins": []
  });
</script>
<script src="https://mediafiles.botpress.cloud/7416bbf4-4223-4490-9169-01c5a3f2ae7a/webchat/config.js" defer></script>